In [85]:
%load_ext autoreload
import datetime
# import ete3
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
import xmlschema

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
from maxes.analyze_xes import PrintXesInfo
import maxes.analyze_sequence
import maxes.serialization.serialize
# import maxes.graphs
import maxes.utils
from maxes.generators.xes_generator1 import XesGenerator1

In [ ]:
import logging
import importlib

def setup_logging():
    # TODO: use __vsc_ipynb_file__
    importlib.reload(logging)
    filename_timestamp_part = datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    filename = f"/vt/md/maxes/maxes/output/logs/notebooks/{filename_timestamp_part}.log"
    logging.basicConfig(
        filename=filename,
        filemode="a",
        format="%(asctime)s %(message)s",
        level=logging.DEBUG)
    logging.info("START")

setup_logging()

In [ ]:
# ET.register_namespace("", "http://www.xes-standard.org")

file_path = "/vt/md/maxes/maxes/data/Activities of daily living of several individuals_1_all/data/edited_hh104_labour.xes/edited_hh104_labour.xes"

printer = PrintXesInfo()
info = printer.print_info(file_path)
info

In [52]:
# dict(printer.info["Events"]["Count per trace"])
printer.xes.traces[0].events['a']

TypeError: list indices must be integers or slices, not str

In [26]:
printer.xes_loader.xml_log

<Element '{http://www.xes-standard.org}log' at 0x124bb8810>

In [38]:
printer.xes.event_attribute_type_map

{'concept:name': 'string',
 'lifecycle:transition': 'string',
 'time:timestamp': 'date',
 'work': 'string'}

In [38]:
# maxes.utils.xml_tag_without_namespace(printer.xes_loader.xml_log.tag).......
# xml_log = printer.xes_loader.xml_log
xml_tree = printer.xes_loader.tree
from collections import Counter

counter = Counter(maxes.utils.xml_tag_without_namespace(element.tag) for element in xml_tree.findall("./*"))
dict(counter)


# counter = Counter(maxes.utils.xml_tag_without_namespace(element.tag) for element in xml_tree.findall("./{http://www.xes-standard.org}trace/*"))
# dict(counter)


# counter = Counter(maxes.utils.xml_tag_without_namespace(element.tag) for element in xml_tree.findall(".//*"))
# dict(counter)

# xml_log.findall(".//*")[0].tag
# maxes.utils.xml_tag_without_namespace("")
# for child in xml_log:
#     print(child.tag)


{'extension': 3, 'global': 2, 'classifier': 1, 'string': 3, 'trace': 43}

In [32]:
xml_tree.find("./{http://www.xes-standard.org}trace/*")

<Element '{http://www.xes-standard.org}string' at 0x133f758f0>

In [54]:
data = xml_tree.findall("./{http://www.xes-standard.org}trace/{http://www.xes-standard.org}event/{http://www.xes-standard.org}date")
date = data[0].attrib["value"]

import datetime
import dateutil

dateutil.parser.isoparse(date)


datetime.datetime(2011, 6, 15, 0, 33, 36, tzinfo=tzoffset(None, 7200))

In [81]:
xml_dates = xml_tree.findall("./{http://www.xes-standard.org}trace/{http://www.xes-standard.org}event/{http://www.xes-standard.org}date[@key='time:timestamp']")

def f(xml_dates):
    for xml_date in xml_dates:
        date = dateutil.parser.isoparse(xml_date.attrib["value"])
        time_zone = date.tzinfo
        time_zone_offset = time_zone.utcoffset(date)
        yield time_zone_offset

# timezones = (dateutil.parser.isoparse(xml_date.attrib["value"]).tzinfo for xml_date in xml_dates)

dict(Counter(f(xml_dates)))

{datetime.timedelta(seconds=7200): 4200}

In [63]:
tz = list(timezones)[0]

In [77]:
date = dateutil.parser.isoparse(xml_dates[0].attrib["value"])

tz.utcoffset(date)

datetime.timedelta(seconds=7200)

In [101]:
import maxes.constants

ET.register_namespace("", "http://www.xes-standard.org")
ET.register_namespace("xes", "http://www.xes-standard.org")
ET.register_namespace("XES", "http://www.xes-standard.org")

# xml_schema_xes_2023 = xmlschema.XMLSchema(schema_path, namespace='http://www.xes-standard.org')

# schema = xmlschema.XMLSchema(maxes.constants.PATH_XES_SCHEMA_2023)
# schema = xmlschema.XMLSchema(maxes.constants.PATH_XES_SCHEMA_2023_EXTENDED)
schema = xmlschema.XMLSchema(maxes.constants.PATH_XES_SCHEMA_2023, namespace='http://www.xes-standard.org')
# schema = xmlschema.XMLSchema(maxes.constants.PATH_XES_SCHEMA_2023_EXTENDED, namespace='http://www.xes-standard.org')

err = None
try:
    schema.validate(file_path)
except xmlschema.XMLSchemaValidationError as error:
    err = error
    print(error.reason)

'xes.creator' attribute not allowed for element


In [104]:
print(err)

failed validating {'xes.version': '1.0', 'xes.creator': 'Fluxicon Disco'} with XsdAttributeGroup(['xes.version', 'xes.features']):

Reason: 'xes.creator' attribute not allowed for element

Schema component:

  <xs:complexType xmlns:xs="http://www.w3.org/2001/XMLSchema" name="LogType">
      <xs:sequence>
          <xs:element name="extension" type="ExtensionType" minOccurs="0" maxOccurs="unbounded" />
          <xs:element name="global" type="GlobalsType" minOccurs="0" maxOccurs="unbounded" />
          <xs:element name="classifier" type="ClassifierType" minOccurs="0" maxOccurs="unbounded" />
          <xs:group ref="AttributableGroup" />
          <xs:element name="trace" type="TraceType" minOccurs="0" maxOccurs="unbounded" />
          <xs:element name="event" type="EventType" minOccurs="0" maxOccurs="unbounded" />
      </xs:sequence>
      <xs:attribute name="xes.version" type="xs:decimal" use="required" />
      <xs:attribute name="xes.features" type="xs:token" />
  </xs:complexTy